# Testing

We will introduce testing in notebooks - you can also develop Python libraries and do more complete testing of those libraries, but this is just a simple introduction.

We will use the library `ipytest` to run tests ([repo](https://github.com/chmp/ipytest/tree/main), [example](https://github.com/chmp/ipytest/blob/main/Example.ipynb)). It needs to be configured to run in notebooks:

In [ ]:
import ipytest
ipytest.autoconfig()

This is already enough to get started. Let's go back to our simple example:

In [ ]:
from numbers import Number

def add(a: Number, b: Number) -> Number:
    """Adds two numbers together, and returns the resulting sum. 
    
    Does not check input types."""
    return a + b

We can now write a test:

In [ ]:
def test_addition():
    assert add(1, 1) == 2

And run the tests:

In [ ]:
ipytest.run('-vv')

**Task**: Change the test function to get an invalid result and see what happens.

This is the basic pattern - write test functions whose names start with `test_` and which use `assert`.

You should normally write at least one test which succeeds and one test which fails. You can use `with pytest.raises(*error*):` to check for errors. For example, in Python we can't add a number and a string:

In [ ]:
1 + "foo"

In [ ]:
import pytest

def test_addition_types():
    with pytest.raises(TypeError):
        add(1, "foo")

In [ ]:
ipytest.run('-vv')

What if we wanted to test for a whole series of values? We don't want to copy-paste our function over and over again. Instead, we can [parameterize our tests](https://docs.pytest.org/en/stable/how-to/parametrize.html#pytest-mark-parametrize-parametrizing-test-functions):

In [ ]:
@pytest.mark.parametrize("a,b,c", [(1, 1, 2), (-1, 1, 0), (-10, -11, -21)])
def test_many_additions(a, b, c):
    assert add(a, b) == c

In [ ]:
ipytest.run('-vv')

**Tasks**: 

* Parameterize another mathematical function
* Write a function to generate many input arguments to define inputs for parameterization

Sometimes tests cannot succeed. For example, the code is broken, or the test relies on an external service which is down. You can use `xfail` and `skip` to mark and handle these cases.

**Task**: 

* Browse the [pytest docs on `xfail` and `skip](https://docs.pytest.org/en/stable/how-to/skipping.html)
* Write and mark a test which is expected to fail
* Write and mark a test which should be skipped

It is quite common to reuse setups or data across tests. We can create these inputs as test fixtures, and reuse them using `pytest.fixture`.

In [ ]:
@pytest.fixture
def some_numbers():
    return (1, 2)

def test_some_additions(some_numbers):
    first, second = some_numbers
    assert add(first, second) == 3

In [ ]:
ipytest.run('-vv')

We also want to run tests which won't break our existing Brightway projects. To do this we will use the `bw2test` decorator. **Make sure** to include this in your tests, otherwise you will make changes to your real data!

In [ ]:
from bw2data.tests import bw2test
import bw2data as bd
import bw2calc as bc

In [ ]:
@pytest.fixture
@bw2test
def biosphere():
    bd.Database("biosphere").write({
        ("biosphere", "1"): {
            "categories": ["things"],
            "code": 1,
            "exchanges": [],
            "name": "an emission",
            "type": "emission",
            "unit": "kg",
        },
        ("biosphere", "2"): {
            "categories": ["things"],
            "code": 2,
            "exchanges": [],
            "type": "emission",
            "name": "another emission",
            "unit": "kg",
        },
    })


@pytest.fixture
def technosphere(biosphere):
    bd.Database("food").write({
        ("food", "1"): {
            "categories": ["stuff", "meals"],
            "code": 1,
            "exchanges": [
                {
                    "amount": 0.5,
                    "input": ("food", "2"),
                    "type": "technosphere",
                    "uncertainty type": 0,
                },
                {
                    "amount": 0.05,
                    "input": ("biosphere", "1"),
                    "type": "biosphere",
                    "uncertainty type": 0,
                },
            ],
            "location": "CA",
            "name": "lunch",
            "type": "process",
            "unit": "kg",
        },
        ("food", "2"): {
            "categories": ["stuff", "meals"],
            "code": 2,
            "exchanges": [
                {
                    "amount": 0.25,
                    "input": ("food", "1"),
                    "type": "technosphere",
                    "uncertainty type": 0,
                },
                {
                    "amount": 0.15,
                    "input": ("biosphere", "2"),
                    "type": "biosphere",
                    "uncertainty type": 0,
                },
            ],
            "location": "CH",
            "name": "dinner",
            "type": "process",
            "unit": "kg",
        },
    })


@pytest.fixture
def lcia(biosphere):
    method = bd.Method(("climate",))
    method.register()
    method.write([(("biosphere", "1"), 10), (("biosphere", "2"), 1000)])

In [ ]:
def test_lcia_score(technosphere, lcia):
    lca = bc.LCA(
        demand={("food", "1"): 1}, 
        method=("climate",)
    )
    lca.lci()
    lca.lcia()
    assert 85 < lca.score < 90

In [ ]:
ipytest.run('-vv')